# KOLMafia RunLog Generator (v1.0)
_A Styling Project by Aaron M. (AKA Captain Scotch, #437479)_

Hey all! This is a wrapper meant to isolate and analyze Ascension logs via Python, formatting everything into a clean Excel wrapper. Originally, this was simply an overlay of [CKB's run log parser in ASH](https://kolmafia.us/showthread.php?22963-RunLogSummary), which is fantastic, but as I'm more of a Python dev this quickly enlarged into a broader project that turned into a personal goal to build a Python log parser. That's this! Contact me in-game or in the Ascension Speed Society discord for any questions.

In [1]:
import requests
import random
import re

# Currently I am leaning on Pandas/Numpy for a few very minor things. 
#   eventually I will probably take out these dependencies so this only
#   requires base Python, since I'll be setting up a web-based parser.

import pandas as pd
import numpy as np

In [2]:
def zoneInformation(mafEnc,mafCom):
    ''' Parser that turns Mafia's encounter dictionary & monster directory into a
          pythonic dictionary that can be accessed for monsters in each zone. Used
          by the parser to figure out what was a noncombat & what was a fight. '''
    
    zInfo = {}
    
    # Parse through all the monster buckets for each zone
    for zText in mafCom.text.split('\n'):
        if   zText in ['', '1']:     continue # handle weird null rows 
        elif zText[0]=='#':          continue # handle comments
        else:
            # Format ======>  zone \t combat% \t monsters 
            zName = zText.split('\t')[0]
            zCPCT = zText.split('\t')[1]
            zMons = zText.split('\t')[2:]
            
            for n, mon in enumerate(zMons):
                # Remove mafia information about boss status & combat weighting (for now) 
                if ':' in mon: 
                    nMon = mon.split(': ')[0]
                    zMons[n] = nMon
                    
            zInfo[zName] = {'cPercent':zCPCT, 'mons':zMons, 'semirare':[], 'clover':[]}
            
    # Parse through all NC encounters in each zone
    for zText in mafEnc.text.split('\n'):
        if   zText in ['', '1']:     continue # handle weird null rows 
        elif zText[0]=='#':          continue # handle comments
        else: 
            # Format ======>  zone \t encounter type \t title 
            zName = zText.split('\t')[0]
            zType = zText.split('\t')[1]
            zEnct = zText.split('\t')[2]
            
            # Just trying to find clovers & semirares
            if   zType == 'CLOVER':   zInfo[zName]['clover']   = zInfo[zName]['clover'] + [zEnct]
            elif zType == 'SEMIRARE': zInfo[zName]['semirare'] = zInfo[zName]['semirare'] + [zEnct]
    
    return zInfo

In [3]:
# To minimize server hits, grab a few files from the Mafia SVN data

#  - mafEncounters = can be used to identify clover & semirares
#  - mafCombats    = combats w/ mafia; can use to figure out combat rate & monster buckets in each zone
#  - mafMonster    = monster names/IDs & stats, as well as drops!
#  - mafItems      = all items in the game, can be used to cross-ref w/ monsterdrops
#  - mafMods       = all modifiers in the game and what they actually do
#  - mafBounty     = all the bounties in the game 

mafEncounters = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/encounters.txt')
mafCombats = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/combats.txt')
mafMonster = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/monsters.txt')
mafItems   = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/items.txt')
mafMods    = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/modifiers.txt')

# Random stuff I might not end up using

mafBounty  = requests.get('https://svn.code.sf.net/p/kolmafia/code/src/data/bounty.txt')
zoneInfo = zoneInformation(mafEncounters,mafCombats)

In [4]:
class newLogParser():
    ''' Giant class that builds a whole dang new log. '''
    
    def __init__(self, runDates = ['20190505','20190506'],
            kolmafDir = '/Users/amcguire/Library/Application Support/KoLmafia/',
            outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/',
            kolName   = 'Captain Scotch',
            zInfo     = {}):
        
        # Set universal variables; directories for inputs & outputs, & the dates of the run you're analyzing
        self.runDates   = runDates
        self.kolmafDir  = kolmafDir
        self.outputDir  = outputDir
        self.kolName    = kolName
        self.zInfo      = zInfo
        
        # Set universal wordbuckets for parsing purposes
        self.musWords = ['Beefiness','Fortitude','Muscleboundness','Strengthliness','Strongness']
        self.mysWords = ['Enchantedness','Magicalness','Mysteriousness','Wizardliness']
        self.moxWords = ['Cheek','Chutzpah','Roguishness','Sarcasm','Smarm']
        
        # initialize a bunch of variables we refer back to & modify
        self.test         = False  # If false, ignores linesToParse
        self.linesToParse = 5000   # Lines to parse; testing function
        
        self.turnSpent    = 0      # How many turns have actually been spent
        self.turnSeen     = -1     # How many "turns" mafia has seen; starts at -1 to bypass valhalla 
        self.lineIndex    = 0      # How many lines it has read
        self.lineIndexPre = 0      # So I can do the line range function; initialize
        self.currDay      = 0      # The current day
        self.newEncounter = False  # This lets the script build its buffer
        self.finalLog     = {}     # The actual final log, w/ "turns seen" as the index
        self.advString    = ''     # Build the adventure string for parsing
        self.finalBuffer  = {}     # The final buffer log, for me to do later parsing. Much more painful.
        self.buffer       = ''     # For storing b/w adventure stuff for later parsing
        
        # Checking mafia's free action counter
        self.freeNC       = 0
        self.freeMafC     = 0
        self.freeMafMiss  = 0
        
        # Initial support for the mayfourth saber friend parsing
        self.saberFriend  = ['']     # What the current Saber Friend is
        self.saberCount   = 0      # How many Saber Friends have been encountered; resets @ 2
        self.saberUsage   = 0      # How many actual fires of the Saber have been done
        
    def preProcessing(self, sLogs):
        ''' In case I want to apply preprocessing in the future. This will
              likely remove bits of the buffer I am 100% sure I do not need. 
              But I haven't really touched the buffer yet, so... '''
        
        for key, day in enumerate(sLogs):
        
            # This new encounter string is *really* messing stuff up. 
            sLogs[key+1] = sLogs[key+1].replace('Encounter: Using the Force','USED THE FORCE')
        
        
        return sLogs 
    
    # ================================================================================
    
    def readSessions(self):
        ''' Subfunction that reads in your sessions into dictionary format '''
        
        dDict = {}
        
        for day, session in enumerate(self.runDates):
            # Python is zero-indexed, so we increment day by 1 here. 
            #   Also replace spaces w/ _ for playername.

            dDict[day+1] = open('{}sessions/{}_{}.txt'.format(self.kolmafDir, self.kolName.replace(" ","_"), session)).read()

            # Remove everything before Valhalla & after freeing Ralph
            if day == 0:
                start = dDict[1].find('Welcome to Valhalla!')
                dDict[1] = '[0] '+dDict[1][start:]

            else:
                end   = dDict[day+1].find('Freeing King Ralph')
                dDict[day+1] = dDict[day+1][:end]
        
        return dDict
    
    # ================================================================================
    
    def resetAdv(self):
        ''' Reset all adventure conditions after parsing a combat '''
        
        self.combround    = -1     # How many rounds the combat went; -1 for NCs
        self.combat       = False  # Is it a combat, or a NC? False for NC, true for C
        self.ncInfo       = {}     # Dictionary storing NC information
        self.mus          = 0      # mus statgain
        self.mys          = 0      # mys statgain
        self.mox          = 0      # mox statgain
        self.hpLoss       = 0      # hp loss
        self.hpGain       = 0      # hp gain
        self.meat         = 0      # meat gain
        self.itemsUsed    = []     # All items *used* in combat
        self.effGained    = []     # All effects gained in combat (tuple; duration after effect name)
        self.items        = []     # All items gained in combat
        self.skills       = []     # All skills cast in combat
        self.currLoct     = "KoL"  # Current location
        self.advTitle     = "???"  # Adventure title; basically the encounter string
        self.freeStatus   = 0      # Did mafia say it cost a turn?
        self.origMonsters = []     # Store the original monster when you macro
    
    # ================================================================================
    
    def parseAdventure(self, advTxt=''):
        ''' Big ol' function for parsing adventures. '''
        for ct, row in enumerate(advTxt.split('\n')): 
            
            # Start by finding the location
            if ']' in row:
                if '[' == row[0]:
                    data = advTxt.split('\n')[ct].split('] ')
                    try:
                        self.currLoct = data[1]
                    except:
                        self.currLoct = '???????? Error ??????'
                        print('weird location thing')
                    try: 
                        line2 = advTxt.split('\n')[ct+1]
                    except:
                        line2 = ''
            
            # Figure out what the adventure title is
            elif 'Encounter:' in row:
                self.advTitle = line2.replace('Encounter: ','')
            
            elif row[0:6] == "Round ":
                self.combround = int(row[6:].split(':')[0])
                
                if '{} casts '.format(self.kolName) in row:
                    self.skills = self.skills + [row.split('{} casts '.format(self.kolName))[1]]
                    
                elif 'hit point' in row:
                    try:
                        if   row.split(' ')[-4] == 'lose': self.hpLoss = self.hpLoss - int(row.split(' ')[-3].replace(',',''))
                        elif row.split(' ')[-4] == 'gain': self.hpGain = self.hpGain + int(row.split(' ')[-3].replace(',',''))
                    
                    except:
                        print('no HP error in HP parsing (round)')
                
                elif '{} uses '.format(self.kolName) in row:
                    self.itemsUsed = self.itemsUsed + [row.split('{} uses '.format(self.kolName))[1]]
                    
                elif 'your opponent becomes a ' in row:
                    self.origMonsters = self.origMonsters + [self.advTitle]
                    self.advTitle = row.split('your opponent becomes a ')[1][:-1]
                    
            
            elif ('You gain ' in row) & (' Meat' in row):
                # adding bug handling via 3bh. thanks 3bh.
                self.meat = self.meat + int(row.split('You gain ')[1].replace('.Huh. You expected more Meat.  They must have spent the rest on weird items to see what they do.','').replace('Meat','').replace(',','').replace('.',''))
                
            elif 'After Battle: You gain' in row:
                statString = row.split(' ')[-1]
                
                if   statString in self.musWords: self.mus = self.mus + int(row.split(' ')[-2].replace(',',''))
                elif statString in self.mysWords: self.mys = self.mys + int(row.split(' ')[-2].replace(',',''))
                elif statString in self.moxWords: self.mox = self.mox + int(row.split(' ')[-2].replace(',',''))
                
                elif 'point' in statString:
                    try:
                        if   row.split(' ')[-4] == 'lose': self.hpLoss = self.hpLoss - int(row.split(' ')[-3].replace(',',''))
                        elif row.split(' ')[-4] == 'gain': self.hpGain = self.hpGain + int(row.split(' ')[-3].replace(',',''))
                    except:
                        print('no HP error in HP parsing')
                        
            elif 'You acquire an item: ' in row:
                self.items = self.items + [row.split('You acquire an item: ')[1]]
                
            elif 'You acquire an effect: ' in row:
                effName = row.split('You acquire an effect: ')[1].split(' (')[0]
                effLen  = row.split('You acquire an effect: ')[1].split(' (')[1][:-1]
                self.effGained = self.effGained + [(effName,int(effLen))]
                
            elif row == 'This combat did not cost a turn':
                self.freeStatus = 1
            
            elif row == 'Took choice 1387/2: &quot;You will go find two friends and meet me here.&quot;':
                self.saberMonster = [self.advTitle]
                self.saberCount =  0
                self.saberUsage += 1
        
        # Now that it's been fully parsed, figure out if it's a combat or an NC. Some 
        #   zones either A: aren't zones or B: aren't easily parsed. Here's some exclusion 
        #   logic as a wraparound to make sure the whole thing works.
        
        exclZones = ['genie summoned monster','Dr. Gordon Stuart\'s Science Tent',
                     'rusty hedge trimmers', 'Daily Dungeon', 'Typical Tavern Cellar',
                     'Lower Chamber', 'The Hedge Maze']
        
        try: 
            # If the round is -1, it means it never found the "round" text, which = NC.
            if self.combround == -1: 
            
                self.ncInfo = {'combat':'N/A','ncTitle':self.advTitle}

                # Determine clover/semirare status.
                if   self.advTitle in self.zInfo[self.currLoct]['clover']:   self.ncInfo.update({'clover':True})
                elif self.advTitle in self.zInfo[self.currLoct]['semirare']: self.ncInfo.update({'semirare':True})

                # For the purposes of this parser, I will treat the zeppelin NC as a clover. 
                #   The SR/Clover adventures have exactly the same name! Rude.
                elif self.advTitle == 'Methinks the Protesters Doth Protest Too Little': 
                    self.ncInfo.update({'clover':True})
            else: 
                # Figure out if the monster is native to that zone.
                monBkt = self.zInfo[self.currLoct]['mons']
                
                if self.advTitle in monBkt: 
                    self.ncInfo.update({'nativeMonster':True})
                else:
                    self.ncInfo.update({'nativeMonster':np.NaN})
                
                if (self.advTitle == self.saberFriend[0]) & (self.ncInfo['nativeMonster'] == True):
                    self.saberCount = self.saberCount + 1
                    
                    # Flush out friends if you have reached the end of your sabers. 
                    if self.saberCount == 2:
                        self.saberCount  = 0
                        self.saberFriend = ['']
        
        # Now apply those exclusion zones.
        except:
                if any(word in self.currLoct for word in exclZones) or (self.advTitle == '???'):
                    pass
                else: 
                    print('Error at {} in zone {}, at "turn" {}'.format(self.advTitle,self.currLoct, self.turnSeen))
                    #print(advTxt)
                    
        advDict = {'rounds': self.combround, 'combat':self.advTitle, 'mus':self.mus, 'mys':self.mys, 'mox':self.mox,
                   'items':self.items, 'location':self.currLoct,'meat':self.meat, 'free?':self.freeStatus, 'skills':self.skills, 
                   'hpGain':self.hpGain,'hpLoss':self.hpLoss, 'itemsUsed':self.itemsUsed, 'effectsGained':self.effGained,
                   'macroFrom':self.origMonsters}
        
        advDict.update(self.ncInfo)
        
        # Reset adv conditions
        self.resetAdv()
        
        return advDict
    
    # ================================================================================
    
    def prepareNewLog(self):
        ''' Alright, let's prepare the damn log then. '''
        
        # Read in your sessions w/ the aforementioned function
        sessionLogs = self.readSessions()
        
        # Use preprocessing to initialize needed variables
        sessionLogs = self.preProcessing(sLogs = sessionLogs)
        self.resetAdv()
        
        # Walk through the files provided for session logs
        for d, sess in enumerate(self.runDates):
            print('Day #{} -- Parsing {}.'.format(d+1, sess))
            
            # When in testing mode, I limited line #s. This disengages the limit if test flag is off.
            if self.test == False:
                self.linesToParse = len(sessionLogs[d+1].split('\n'))
                
                
            # Enumerate through all lines of your session logs
            for ct, row in enumerate(sessionLogs[d+1].split('\n')[0:self.linesToParse]):
                self.lineIndex += 1
                
                if row == '':
                    self.newEncounter = True
                elif row[0] == '[':
                    self.newEncounter = False

                    # ==========================================================
                    # -- This is where it adds the actual new "turn" to the buffer.
                    # --   Each turn is a dictionary that can be easily converted 
                    # --   into a pandas DF. The buffer is all the stuff that 
                    # --   happens between adventures; I'll use this to figure out
                    # --   resource management stuff. Later. 
                    # ==========================================================

                    self.finalBuffer[self.turnSeen] = {
                        'buffer':    self.buffer,
                        'turnSpent': self.turnSpent,
                        'lineRange': [self.lineIndexPre,self.lineIndex]
                    }

                    aDict = self.parseAdventure(advTxt = self.advString)

                    self.finalLog[self.turnSeen] = {
                        'turnSpent': self.turnSpent,
                        'day':d+1,
                        'lineRange': [self.lineIndexPre,self.lineIndex]
                    }

                    self.finalLog[self.turnSeen].update(aDict)

                    self.lineIndexPre = self.lineIndex
                    self.buffer = ''
                    self.advString = ''
                    self.turnSeen  += 1
                    self.turnSpent = row[(row.find('[')+1):row.find(']')]

                if self.newEncounter == True:
                    # Update the buffer if it's buffer time
                    self.buffer = self.buffer + row

                elif self.newEncounter == False:
                    # Create the adv string for parsing it
                    self.advString = self.advString + '\n' + row

        # Apply all post-facto processing 
        self.postProcessing()
        
        # Make the final CSV a bit nicer
        colOrder = ['day','turnSpent','advUsed','location','combat', 'free?', 
            'items','meat', 'rounds', 'mox', 'mus', 'mys','effectsGained','clover', 
            'nativeMonster','hpGain', 'hpLoss', 'ncTitle',  'semirare', 'itemsUsed',
            'skills', 'crafting', 'lineRange','freeNC','macroFrom']
        
        pLog = pd.DataFrame(self.finalLog).T.loc[:,colOrder]
        pLog.to_csv('{} -- {}d{}advs.csv'.format(self.kolName,len(self.runDates),self.turnSpent))
        
        return pLog
    
    # ================================================================================
    
    def postProcessing(self):
        
        # Remove turn -1 and turnseen = 0 from the log
        self.finalLog.pop(-1)
        self.finalLog.pop(0)
        
        # Add a new 'free' counter that references whether the task took an adv. 
        #   Uses mafia's as base & prints how many "free" items mafia missed.
        
        for t in self.finalLog:
            try: 
                # For free actions, the next row's turnSpent will be equal.
                if self.finalLog[t]['turnSpent'] == self.finalLog[t+1]['turnSpent']:
                    if self.finalLog[t]['rounds'] == -1:
                        self.freeNC += 1
                        self.finalLog[t]['free?'] = 1
                    elif self.finalLog[t]['free?'] == 1:
                        self.freeMafC += 1
                    else:
                        self.freeMafMiss += 1
                        self.finalLog[t]['free?'] = 1    
            except:
                pass
        
        # Report back on how Mafia did!
        print('\nMafia correctly captured {} free combats. It missed {} free/skipped NCs & {} free combats.'.format(self.freeMafC, self.freeNC, self.freeMafMiss))
        
        # How many times did we use the saber?
        print('\nYou used the saber {} times. Embrace the force!'.format(self.saberUsage))
        
        # Rename/refactor crafting so that it's easier to summarize.
        for t in self.finalLog:
            if self.finalLog[t]['location'][0:4] == 'Cook':
                self.finalLog[t]['crafting'] = self.finalLog[t]['location']
                self.finalLog[t]['location'] = 'Crafting Adventure'
            elif self.finalLog[t]['location'][0:4] == 'Mix ':
                self.finalLog[t]['crafting'] = self.finalLog[t]['location']
                self.finalLog[t]['location'] = 'Crafting Adventure'
            # Eventually put smithing here, I don't know how that generates in mafia logs tho.
            
        # Rename/refactor lair stat & damage tests
        elementalTests = ['Coldest', 'Hottest', 'Stinkiest', 'Sleaziest', 'Spookiest']
        statTests = ['Smoothest', 'Strongest', 'Smartest']
        
        for t in self.finalLog:
            if any(test+' Adventurer Contest'==self.finalLog[t]['location'] for test in elementalTests):
                self.finalLog[t]['location'] = '[Element]est Adventurer Contest'
                
            elif any(test+' Adventurer Contest'==self.finalLog[t]['location'] for test in statTests):
                self.finalLog[t]['location'] = '[Offstat]est Adventurer Contest'
        
        # Rename/refactor the Tavern Cellar, Daily Dungeon, & Hedge Maze
        for t in self.finalLog:
            if 'The Typical Tavern Cellar' in self.finalLog[t]['location']:
                self.finalLog[t]['location'] = 'The Typical Tavern Cellar'
            elif 'Daily Dungeon' in self.finalLog[t]['location']:
                self.finalLog[t]['location'] = 'The Daily Dungeon'
            elif 'Hedge Maze' in self.finalLog[t]['location']:
                self.finalLog[t]['location'] = 'The Hedge Maze'
            elif 'Lower Chamber' in self.finalLog[t]['location']:
                self.finalLog[t]['location'] = 'The Lower Chamber'
        
        # Add a flag for free/skipped NCs so they're easier to filter out 
        for t in self.finalLog:
            if (self.finalLog[t]['rounds'] == -1) & self.finalLog[t]['free?']:
                self.finalLog[t]['freeNC'] = 1
        
        # Add a variable that counts the # of adventures used by that row
        for t in self.finalLog:
            try:
                self.finalLog[t]['advUsed'] = int(self.finalLog[t+1]['turnSpent']) - int(self.finalLog[t]['turnSpent'])
            except:
                self.finalLog[t]['advUsed'] = 1
        
newLog = newLogParser(runDates=['20190506','20190507'],zInfo=zoneInfo).prepareNewLog()

Day #1 -- Parsing 20190506.
Day #2 -- Parsing 20190507.
Error at The Mirror in the Tower has the View that is True in zone Tower Level 4, at "turn" 740

Mafia correctly captured 163 free combats. It missed 84 free/skipped NCs & 10 free combats.

You used the saber 10 times. Embrace the force!


In [5]:
newLog.head()

,day,turnSpent,advUsed,location,combat,free?,items,meat,rounds,mox,...,hpGain,hpLoss,ncTitle,semirare,itemsUsed,skills,crafting,lineRange,freeNC,macroFrom
1,1,1,1,Guano Junction,N/A,0,"[sonar-in-a-biscuit, sonar-in-a-biscuit]",0,-1,0,...,0,0,How I Wonder What You're At,NaN,[],[],NaN,"[871, 914]",NaN,[]
2,1,2,1,The Haunted Kitchen,skullery maid,0,"[blood bag, bottle of popskull, Skullery Maid'...",179,4,12,...,105,-1,NaN,NaN,[],"[SING ALONG!, DARK FEAST!]",NaN,"[914, 943]",NaN,[]
3,1,3,1,The Outskirts of Cobb's Knob,Knob Goblin Assistant Chef,0,"[chef's hat, Gathered Meat-Clip]",27,2,8,...,0,0,NaN,NaN,[],[],NaN,"[943, 963]",NaN,[]
4,1,4,0,The Outskirts of Cobb's Knob,Sub-Assistant Knob Mad Scientist,1,[],0,1,0,...,0,0,NaN,NaN,[],[ENSORCEL!],NaN,"[963, 976]",NaN,[]
5,1,4,0,The Haunted Kitchen,sausage goblin,1,"[magical sausage casing, Cobb's Knob map]",378,3,16,...,12,-1,NaN,NaN,[],"[SING ALONG!, CHILL OF THE TOMB!]",NaN,"[976, 1008]",NaN,[]


In [6]:
def resourcesByZone(log):
    ''' Extremely basic function to extract resource usage by zone.
        Currently leans a lot on Pandas; it doesn't really need to,
        and I will be moving it into a function within the bigger 
        class at some point soon. '''
    
    # Remove free NCs first
    locSumm = log.loc[log['freeNC']!=1,:].groupby('location').agg({'free?':[sum,len],'advUsed':[sum]})

    locSumm.columns = ['freeTurns','allTurns','actualAdvs']
    
    banishSkills = ['ASDON MARTIN: SPRING-LOADED FRONT BUMPER!',
                  'REFLEX HAMMER!', 'KGB TRANQUILIZER DART!',
                  'THROW LATTE ON OPPONENT!','BALEFUL HOWL!',
                   'SNOKEBOMB!']
    
    banishItems = ['the Louder Than Bomb']
    
    sniffSkills  = ['OFFER LATTE TO OPPONENT!','PERCEIVE SOUL!']
    
    freeKills    = ['ASDON MARTIN: MISSILE LAUNCHER!','CHEST X-RAY!',
                    'GINGERBREAD MOB HIT!','SHATTERING PUNCH!']
    
    freeRunSkills = ['RETURN!']
    freeRunItems = ['the short writ of habeas corpus']
    
    locDict = {}
    
    for location in locSumm.index:
        locFull = log.loc[log['location']==location,:]
        
        # Create a list of all items used in the zone, for smoke bombs, enamorangs, & odor extractors.
        locItems = list(locFull['itemsUsed'].apply(pd.Series).stack().reset_index(drop=True))
        
        # Create a list of all skills used in the zone, for sniffs & banishes.
        locSkills = list(locFull['skills'].apply(pd.Series).stack().reset_index(drop=True))
        
        # Count # of sausage goblins in the zone
        gobCount = sum([x=='sausage goblin' for x in locFull['combat'].apply(pd.Series).stack().reset_index(drop=True)])
        
        # Test against the list for skills
        bCount = sum([x in banishSkills for x in locSkills])
        sCount = sum([x in sniffSkills  for x in locSkills])
        fkCount = sum([x in freeKills   for x in locSkills])
        macrCount = sum([x=='MACROMETEORITE!' for x in locSkills])
        sabrCount = sum(['USE THE FORCE' in x for x in locSkills])
        ensorcelCt = sum(['ENSORCEL!' in x for x in locSkills])
        frCount = sum([x in freeRunSkills for x in locSkills])
        odorCount = sum(['TRANSCENDENT OLFACTION!' in x for x in locSkills])
        
        # ... and test against the list for items
        gropCount = sum(['the green smoke bomb' in x for x in locItems])
        enamCount = sum(['the LOV Enamorang!' in x for x in locItems])
        odorCount = odorCount + sum(['the odor extractor!' in x for x in locItems])
        
        # ... and add in power pill freekills, and item banishes
        fkCount = fkCount + sum(['the power pill' in x for x in locItems])
        for frSource in freeRunItems: 
            frCount = frCount + sum([frSource in x for x in locItems])
        for bSource in banishItems:
            bCount = bCount + sum([bSource in x for x in locItems])
        
        locDict[location] = {'macros':macrCount,
                             'freeKills':fkCount,
                             'banishes':bCount,
                             'sniffs':sCount,
                             'GSBs':gropCount,
                             'enamorangs':enamCount,
                             'odorExtractors':odorCount,
                             'goblinCount':gobCount,
                             'saberCount':sabrCount,
                             'ensorcels':ensorcelCt,
                             'freeRuns':frCount}

    finalTable = locSumm.loc[:,['freeTurns','actualAdvs']].join(pd.DataFrame(locDict).T).sort_values(by='actualAdvs',ascending=False)
    
    return finalTable.loc[:,['actualAdvs','freeTurns','banishes','GSBs','freeRuns','freeKills','saberCount','goblinCount','ensorcels','macros','odorExtractors','sniffs','enamorangs']]

resourcesByZone(newLog)

,actualAdvs,freeTurns,banishes,GSBs,freeRuns,freeKills,saberCount,goblinCount,ensorcels,macros,odorExtractors,sniffs,enamorangs
location,,,,,,,,,,,,,
The Battlefield (Frat Uniform),41,11,4,0,0,2,5,0,0,0,1,2,1
The Penultimate Fantasy Airship,22,4,4,0,0,0,0,0,0,0,0,0,0
The Black Forest,17,0,0,0,0,0,0,0,0,0,0,0,0
The Dark Heart of the Woods,16,0,0,0,0,0,0,0,0,0,0,0,0
The Dark Neck of the Woods,15,0,0,0,0,0,0,0,0,0,0,0,0
"The Arid, Extra-Dry Desert",15,6,0,0,0,5,0,1,0,0,0,0,0
"The Shore, Inc. Travel Agency",15,0,0,0,0,0,0,0,0,0,0,0,0
The Copperhead Club,14,6,4,0,0,0,0,1,0,0,0,2,0
The Smut Orc Logging Camp,13,0,0,0,0,0,0,0,0,1,0,0,0


In [7]:
# ======================== 
# ------ TO-DO LIST ------
# ========================
#   - add parsing of daily iotm-based resources (bastille, LOV choices, daycare, etc).
#   - track player stats through the run & store when levelups occur & when thresholds reached.
#   - direct reporting of GSBs failed (technically can be calculated from resource table).
#   - consumption parsing & actual turns generated via diet. 
#   - also turns leftover @ end of run.
#   - code up zones per quest, then map out a run's specific route map
#   - summary sheet w/ run conditions; # of clovers, vote monsters, bounties, etc
#   - where were your asdons

In [8]:
oneDay = newLogParser(runDates=['20190405'],zInfo=zoneInfo).prepareNewLog()
resourcesByZone(oneDay).to_csv('oneDayLocationSummary.csv')

Day #1 -- Parsing 20190405.
Error at The Mirror in the Tower has the View that is True in zone Tower Level 4, at "turn" 721
Error at You acquire an item: Instant Karma in zone Freeing King Ralph, at "turn" 727
Error at play Gift Card (41) in zone Deck of Every Card, at "turn" 728
Error at play Island (35) in zone Deck of Every Card, at "turn" 729
Error at play Ancestral Recall (40) in zone Deck of Every Card, at "turn" 730

Mafia correctly captured 96 free combats. It missed 78 free/skipped NCs & 0 free combats.

You used the saber 0 times. Embrace the force!


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [11]:
newLog = newLogParser(runDates=['20190713','20190714'],zInfo=zoneInfo).prepareNewLog()
resourcesByZone(newLog).to_csv('two466LocationSummary.csv')

Day #1 -- Parsing 20190713.
Error at the God Lobster in zone God Lobster, at "turn" 1
Error at Granted a Boon in zone God Lobster, at "turn" 2
Error at the God Lobster in zone God Lobster, at "turn" 12
Error at Granted a Boon in zone God Lobster, at "turn" 13
Error at the God Lobster in zone God Lobster, at "turn" 16
Error at Granted a Boon in zone God Lobster, at "turn" 17
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 85
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 86
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 87
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 88
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 89
Error at Eldritch Tentacle in zone Evoke Eldritch Horror, at "turn" 95
Day #2 -- Parsing 20190714.
Error at The Mirror in the Tower has the View that is True in zone Tower Level 4, at "turn" 729

Mafia correctly captured 146 free combats. It missed 113 free/skipped NCs & 11 free combats.

You

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [15]:
newLog = newLogParser(runDates=['20190511','20190512'],zInfo=zoneInfo).prepareNewLog()
resourcesByZone(newLog).to_csv('two483LocationSummary.csv')

Day #1 -- Parsing 20190511.
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing (round)
Day #2 -- Parsing 20190512.
Error at Island #1, Who Are You? in zone Sailing the PirateRealm Seas, at "turn" 650
Error at Avast, a Mast! in zone Sailing the PirateRealm Seas, at "turn" 651
Error at Stormy Weather in zone Sailing the PirateRealm Seas, at "turn" 652
Error at Who Pirates the Pirates? in zone Sailing the PirateRealm Seas, at "turn" 653
Error at Land Ho! in zone Sailing the PirateRealm Seas, at "turn" 654
Error at The Mirror in the Tower has the View that is True in zone Tower Level 4, at "turn" 762

Mafia correctly captured 172 free combats. It missed 103 free/skipped NCs & 11 free combats.

You used the saber 10 times. Embrace the force!


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [23]:
scotch1day = newLogParser(runDates=['20190512'],zInfo=zoneInfo,kolName='Captain Scotch').prepareNewLog()
resourcesByZone(scotch1day)

Day #1 -- Parsing 20190512.
Error at Island #1, Who Are You? in zone Sailing the PirateRealm Seas, at "turn" 1
Error at Stormy Weather in zone Sailing the PirateRealm Seas, at "turn" 621
Error at An Opportunity for Dastardly Do in zone Sailing the PirateRealm Seas, at "turn" 622
Error at Avast, a Mast! in zone Sailing the PirateRealm Seas, at "turn" 623
Error at Land Ho! in zone Sailing the PirateRealm Seas, at "turn" 624
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
Error at The Mirror in the Tower has the View that is True in zone Tower Level 4, at "turn" 734
Error at Took choice 1181/1: Examine the shrink ray in zone Your Witchess Set, at "turn" 738
Error at Took choice 1181/1: Examine the shrink ray in zone Your Witchess Set, at "turn" 739
Error at Took choice 1181/1: Examine the shrink ray in zone Your Witchess Set, at "turn" 740
Error at Took choice 1181/1: Examine the shrink ray in zone Your Witchess Set, at "turn" 741
Error at Took choice 1181/1:

,actualAdvs,freeTurns,banishes,GSBs,freeKills,saberCount,goblinCount,ensorcels,macros,odorExtractors,sniffs,enamorangs
location,,,,,,,,,,,,
The Battlefield (Frat Uniform),42,6,2,0,0,3,1,0,0,1,2,1
The Penultimate Fantasy Airship,24,3,2,0,0,0,1,0,0,0,0,0
Lair of the Ninja Snowmen,21,6,1,7,0,0,0,0,0,0,2,0
The Spooky Forest,21,4,1,0,0,0,1,0,0,0,0,0
The Black Forest,19,0,0,0,0,0,0,0,0,0,0,0
"The Arid, Extra-Dry Desert",18,2,0,0,2,0,0,0,0,0,0,0
The Themthar Hills,16,0,0,0,0,0,0,0,0,0,0,0
The Smut Orc Logging Camp,16,0,0,0,0,0,0,0,0,0,0,0
The Castle in the Clouds in the Sky (Ground Floor),15,0,0,0,0,0,0,0,0,0,0,0


In [17]:
lyft1day = newLogParser(runDates=['20190512'],zInfo=zoneInfo,kolName='Lyft').prepareNewLog()
resourcesByZone(lyft1day)

Day #1 -- Parsing 20190512.
Error at Island #1, Who Are You? in zone Sailing the PirateRealm Seas, at "turn" 1
Error at Who Pirates the Pirates? in zone Sailing the PirateRealm Seas, at "turn" 564
Error at Stormy Weather in zone Sailing the PirateRealm Seas, at "turn" 565
Error at The Ship is Wrecked in zone Sailing the PirateRealm Seas, at "turn" 566
Error at Who Pirates the Pirates? in zone Sailing the PirateRealm Seas, at "turn" 567
Error at High Tide, Low Morale in zone Sailing the PirateRealm Seas, at "turn" 568
Error at Land Ho! in zone Sailing the PirateRealm Seas, at "turn" 569
Error at The Mirror in the Tower has the View that is True in zone Tower Level 4, at "turn" 710

Mafia correctly captured 121 free combats. It missed 77 free/skipped NCs & 5 free combats.

You used the saber 5 times. Embrace the force!


,actualAdvs,freeTurns,banishes,GSBs,freeKills,saberCount,goblinCount,ensorcels,macros,odorExtractors,sniffs,enamorangs
location,,,,,,,,,,,,
The Battlefield (Frat Uniform),41,3,1,0,0,2,0,0,2,1,1,1
The Penultimate Fantasy Airship,25,2,1,0,0,0,1,0,0,0,3,0
"The Arid, Extra-Dry Desert",20,3,0,0,3,0,0,0,0,0,0,0
The Themthar Hills,18,0,0,0,0,0,0,0,0,0,0,0
The Copperhead Club,15,3,0,0,0,0,3,0,0,0,0,0
The Smut Orc Logging Camp,15,0,0,0,0,0,0,0,0,0,0,0
The Black Forest,15,0,0,0,0,0,0,0,0,0,0,0
"The Shore, Inc. Travel Agency",15,0,0,0,0,0,0,0,0,0,0,0
8-Bit Realm,13,3,2,0,0,1,0,0,0,1,2,0


In [25]:
lyft2day = newLogParser(runDates=['20190511','20190512'],zInfo=zoneInfo,kolName='Lyft').prepareNewLog()
resourcesByZone(lyft2day).to_csv('aenAllocate.csv')

Day #1 -- Parsing 20190511.
no HP error in HP parsing (round)
no HP error in HP parsing (round)
Error at Island #1, Who Are You? in zone Sailing the PirateRealm Seas, at "turn" 289
Error at A Sea Monster! in zone Sailing the PirateRealm Seas, at "turn" 290
Error at A Sea Monster! in zone Sailing the PirateRealm Seas, at "turn" 291
Error at A Sea Monster! in zone Sailing the PirateRealm Seas, at "turn" 292
Error at The Starboard is Bare in zone Sailing the PirateRealm Seas, at "turn" 293
Error at Land Ho! in zone Sailing the PirateRealm Seas, at "turn" 294
Day #2 -- Parsing 20190512.
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP pa

In [33]:
tes2day = newLogParser(runDates=['20190510','20190511'],zInfo=zoneInfo,kolName='the erosionseeker').prepareNewLog()
resourcesByZone(tes2day).to_csv('aenAllocate.csv')

Day #1 -- Parsing 20190510.
weird location thing
weird location thing
Error at You have cast your votes in zone ???????? Error ??????, at "turn" 2
Error at You acquire loaf of soda bread (3) in zone Cook 3 wad of dough + 3 soda water, at "turn" 80
no HP error in HP parsing (round)
Error at You acquire loaf of soda bread (20) in zone Cook 20 wad of dough + 20 soda water, at "turn" 189
Error at Island #1, Who Are You? in zone Sailing the PirateRealm Seas, at "turn" 213
Error at Avast, a Mast! in zone Sailing the PirateRealm Seas, at "turn" 214
Error at A Sea Monster! in zone Sailing the PirateRealm Seas, at "turn" 215
Error at A Sea Monster! in zone Sailing the PirateRealm Seas, at "turn" 216
Error at A Sea Monster! in zone Sailing the PirateRealm Seas, at "turn" 217
Error at Land Ho! in zone Sailing the PirateRealm Seas, at "turn" 219
Day #2 -- Parsing 20190511.
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP pars

In [24]:
aen2day = newLogParser(runDates=['20190511','20190512'],zInfo=zoneInfo,kolName='Aenimus').prepareNewLog()
resourcesByZone(aen2day).to_csv('aenAllocate.csv')

Day #1 -- Parsing 20190511.
no HP error in HP parsing (round)
no HP error in HP parsing (round)
no HP error in HP parsing (round)
no HP error in HP parsing (round)
Day #2 -- Parsing 20190512.
Error at Island #1, Who Are You? in zone Sailing the PirateRealm Seas, at "turn" 697
Error at Avast, a Mast! in zone Sailing the PirateRealm Seas, at "turn" 698
Error at Avast, a Mast! in zone Sailing the PirateRealm Seas, at "turn" 699
Error at An Opportunity for Dastardly Do in zone Sailing the PirateRealm Seas, at "turn" 700
Error at Stormy Weather in zone Sailing the PirateRealm Seas, at "turn" 701
Error at The Ship is Wrecked in zone Sailing the PirateRealm Seas, at "turn" 702
Error at Land Ho! in zone Sailing the PirateRealm Seas, at "turn" 703
Error at The Mirror in the Tower has the View that is True in zone Tower Level 4, at "turn" 716

Mafia correctly captured 173 free combats. It missed 112 free/skipped NCs & 14 free combats.

You used the saber 10 times. Embrace the force!


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [35]:
scotchLastRun = newLogParser(runDates=['20190513','20190514'],zInfo=zoneInfo,kolName='Captain Scotch').prepareNewLog()
resourcesByZone(scotchLastRun).to_csv('scotchAllocate.csv')

Day #1 -- Parsing 20190513.
Day #2 -- Parsing 20190514.
Error at Island #1, Who Are You? in zone Sailing the PirateRealm Seas, at "turn" 623
Error at Avast, a Mast! in zone Sailing the PirateRealm Seas, at "turn" 624
Error at A Sea Monster! in zone Sailing the PirateRealm Seas, at "turn" 625
Error at An Opportunity for Dastardly Do in zone Sailing the PirateRealm Seas, at "turn" 626
Error at An Opportunity for Dastardly Do in zone Sailing the PirateRealm Seas, at "turn" 627
Error at Land Ho! in zone Sailing the PirateRealm Seas, at "turn" 628
Error at The Mirror in the Tower has the View that is True in zone Tower Level 4, at "turn" 755

Mafia correctly captured 189 free combats. It missed 81 free/skipped NCs & 12 free combats.

You used the saber 9 times. Embrace the force!


In [44]:
lyftStandard = newLogParser(runDates=['20190209','20190210'],zInfo=zoneInfo,kolName='Lyft').prepareNewLog()
resourcesByZone(lyftStandard).to_csv('lyftStandard.csv')

Day #1 -- Parsing 20190209.
Error at the God Lobster in zone God Lobster, at "turn" 1
Error at Granted a Boon in zone God Lobster, at "turn" 2
Error at the God Lobster in zone God Lobster, at "turn" 4
Error at Granted a Boon in zone God Lobster, at "turn" 5
Error at the God Lobster in zone God Lobster, at "turn" 6
Error at Granted a Boon in zone God Lobster, at "turn" 7
Error at Eldritch Tentacle in zone Evoke Eldritch Horror, at "turn" 37
Day #2 -- Parsing 20190210.
Error at Eldritch Tentacle in zone Evoke Eldritch Horror, at "turn" 374
Error at If You Could Only See in zone tonic djinn, at "turn" 408
Error at > queue  in zone lynyrd snare, at "turn" 529
Error at the God Lobster in zone God Lobster, at "turn" 676
Error at Granted a Boon in zone God Lobster, at "turn" 677
Error at the God Lobster in zone God Lobster, at "turn" 678
Error at Granted a Boon in zone God Lobster, at "turn" 679
Error at The Mirror in the Tower has the View that is True in zone Tower Level 4, at "turn" 721

M

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [10]:
bobGold = newLogParser(runDates=['20190503'],zInfo=zoneInfo,kolName='monsieur bob').prepareNewLog()
resourcesByZone(bobGold).to_csv('bobGold.csv')

Day #1 -- Parsing 20190503.
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing
no HP error in HP parsing (round)
no HP error in HP parsing (round)
Error at Island #1, Who Are You? in zone Sailing the PirateRealm Seas, at "turn" 533
Error at Who Pirates the Pirates? in zone Sailing the PirateRealm Seas, at "turn" 534
Error at Who Pirates the Pirates? in zone Sailing the PirateRealm Seas, at "turn" 535
Error at Avast, a Mast! in zone Sailing the PirateRealm Seas, at "turn" 536
Error at A Sea Monster! in zone Sailing the PirateRealm Seas, at "turn" 537
Error at Land Ho! in zone Sailing the PirateRealm Seas, at "turn" 538
Error at The Mirror in the Tower has the View that is True in zone Tower Level 4, at "turn" 679

Mafia correctly captured 112 free combats. It missed 73 free/skipped NCs & 5 free combats.

Yo

In [29]:
aenUnrestricted = newLogParser(runDates=['20190527','20190528'],zInfo=zoneInfo,kolName='Aenimus').prepareNewLog()
resourcesByZone(aenUnrestricted).to_csv('aenunrestricted.csv')
resourcesByZone(aenUnrestricted)

Day #1 -- Parsing 20190527.
Error at play XVI - The Tower (47) in zone Deck of Every Card, at "turn" 1
Error at play Sheep (61) in zone Deck of Every Card, at "turn" 2
Error at play 1952 Mickey Mantle (58) in zone Deck of Every Card, at "turn" 3
Error at the God Lobster in zone God Lobster, at "turn" 7
Error at Granted a Boon in zone God Lobster, at "turn" 8
Error at the God Lobster in zone God Lobster, at "turn" 15
Error at Granted a Boon in zone God Lobster, at "turn" 16
Error at the God Lobster in zone God Lobster, at "turn" 17
Error at Granted a Boon in zone God Lobster, at "turn" 18
Error at Took choice 1181/1: Examine the shrink ray in zone Your Witchess Set, at "turn" 38
Error at Took choice 1181/1: Examine the shrink ray in zone Your Witchess Set, at "turn" 61
Error at Took choice 1181/1: Examine the shrink ray in zone Your Witchess Set, at "turn" 62
Error at Eldritch Tentacle named Blrgggggrrgglssssblllrrgggh in zone Evoke Eldritch Horror, at "turn" 66
Error at Took choice 118

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,actualAdvs,freeTurns,banishes,GSBs,freeRuns,freeKills,saberCount,goblinCount,ensorcels,macros,odorExtractors,sniffs,enamorangs
location,,,,,,,,,,,,,
The Penultimate Fantasy Airship,30,4,1,0,0,0,0,1,0,0,0,0,0
The Battlefield (Frat Uniform),29,9,1,0,0,6,2,0,0,0,0,1,0
"The Arid, Extra-Dry Desert",24,1,0,0,0,1,0,0,0,0,0,0,0
"The Shore, Inc. Travel Agency",18,0,0,0,0,0,0,0,0,0,0,0,0
The Haunted Billiards Room,14,0,0,0,0,0,0,0,0,0,0,0,0
The Smut Orc Logging Camp,14,0,0,0,0,0,0,0,0,0,0,0,0
The Spooky Forest,12,6,2,0,1,0,0,1,0,0,0,0,0
The Black Forest,12,0,0,0,0,0,0,0,0,0,0,0,0
[Offstat]est Adventurer Contest,11,1,1,0,0,0,0,0,0,1,0,0,0


In [9]:
lyft2CRS = newLogParser(runDates=['20190810','20190811'],zInfo=zoneInfo,kolName='Lyft').prepareNewLog()
resourcesByZone(lyft2CRS).to_csv('lyft2CRS.csv')
resourcesByZone(lyft2CRS)

Day #1 -- Parsing 20190810.
Error at the God Lobster in zone God Lobster, at "turn" 11
Error at Granted a Boon in zone God Lobster, at "turn" 12
Error at the God Lobster in zone God Lobster, at "turn" 13
Error at Granted a Boon in zone God Lobster, at "turn" 14
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 71
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 72
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 73
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 74
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 75
Error at Eldritch Tentacle in zone Evoke Eldritch Horror, at "turn" 85
Error at the God Lobster in zone God Lobster, at "turn" 256
Error at Granted a Boon in zone God Lobster, at "turn" 257
Day #2 -- Parsing 20190811.
no HP error in HP parsing (round)
Error at Spawn of Wally in zone Infernal Seal Ritual, at "turn" 389
Error at Spawn of Wally in zone Infernal Seal Ritual, at "turn" 390
Error at Spawn of Wa

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,actualAdvs,freeTurns,banishes,GSBs,freeRuns,freeKills,saberCount,goblinCount,ensorcels,macros,odorExtractors,sniffs,enamorangs
location,,,,,,,,,,,,,
The Battlefield (Frat Uniform),93,5,0,0,0,0,5,0,0,4,2,1,0
The Penultimate Fantasy Airship,19,7,0,0,0,1,0,6,0,0,1,0,0
"The Arid, Extra-Dry Desert",16,4,0,0,0,3,0,0,0,0,0,0,0
"The Shore, Inc. Travel Agency",15,0,0,0,0,0,0,0,0,0,0,0,0
The Middle Chamber,12,2,1,0,0,1,0,0,0,0,1,0,0
The Black Forest,12,0,0,0,0,0,0,0,0,0,0,0,0
The Hidden Bowling Alley,10,12,1,0,0,0,0,0,0,0,1,0,0
The Lower Chamber,10,0,0,0,0,0,0,0,0,0,0,0,0
The Copperhead Club,10,5,3,0,0,0,0,2,0,0,0,0,0


In [10]:
lyft2CRS = newLogParser(runDates=['20190811','20190812'],zInfo=zoneInfo,kolName='Lyft').prepareNewLog()
resourcesByZone(lyft2CRS).to_csv('SClyft2CRS.csv')
resourcesByZone(lyft2CRS)

Day #1 -- Parsing 20190811.
Error at the God Lobster in zone God Lobster, at "turn" 14
Error at Granted a Boon in zone God Lobster, at "turn" 15
Error at the God Lobster in zone God Lobster, at "turn" 16
Error at Granted a Boon in zone God Lobster, at "turn" 17
Error at the God Lobster in zone God Lobster, at "turn" 25
Error at Granted a Boon in zone God Lobster, at "turn" 26
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 66
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 67
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 68
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 69
Error at hermetic seal in zone Infernal Seal Ritual, at "turn" 70
Error at Eldritch Tentacle in zone Evoke Eldritch Horror, at "turn" 80
Day #2 -- Parsing 20190812.
Error at the God Lobster in zone God Lobster, at "turn" 322
Error at Granted a Boon in zone God Lobster, at "turn" 323
Error at Spawn of Wally in zone Infernal Seal Ritual, at "turn" 331
Error

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,actualAdvs,freeTurns,banishes,GSBs,freeRuns,freeKills,saberCount,goblinCount,ensorcels,macros,odorExtractors,sniffs,enamorangs
location,,,,,,,,,,,,,
The Battlefield (Frat Uniform),85,15,0,0,0,6,7,0,0,4,1,1,0
The Penultimate Fantasy Airship,19,9,0,5,0,1,0,5,0,0,1,1,0
"The Arid, Extra-Dry Desert",18,2,0,0,0,0,0,0,0,0,0,0,0
The Middle Chamber,10,1,0,0,0,0,0,1,0,0,1,0,0
The Lower Chamber,10,0,0,0,0,0,0,0,0,0,0,0,0
The Black Forest,10,0,0,0,0,0,0,0,0,0,0,0,0
McMillicancuddy's Barn,9,2,0,0,0,0,0,2,0,0,0,0,0
The Hidden Bowling Alley,8,11,0,0,0,0,0,0,0,0,1,0,0
A-Boo Peak,8,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
ssbb2CRS = newLogParser(runDates=['20190812','20190813'],zInfo=zoneInfo,kolName='SSBBHax').prepareNewLog()
resourcesByZone(ssbb2CRS).to_csv('ssbb2CRS.csv')
resourcesByZone(ssbb2CRS)

Day #1 -- Parsing 20190812.
Error at Preference lastEncounter changed from Welcome to FantasyRealm to the God Lobster in zone God Lobster, at "turn" 2
Error at Preference lastEncounter changed from the God Lobster to Granted a Boon in zone God Lobster, at "turn" 3
Error at Preference lastEncounter changed from Upgrade Your May the Fourth Cosplay Saber to the God Lobster in zone God Lobster, at "turn" 9
Error at Preference lastEncounter changed from the God Lobster to Granted a Boon in zone God Lobster, at "turn" 10
Error at Preference lastEncounter changed from Granted a Boon to the God Lobster in zone God Lobster, at "turn" 11
Error at Preference lastEncounter changed from the God Lobster to Granted a Boon in zone God Lobster, at "turn" 12
Error at Preference lastEncounter changed from giant swarm of ghuol whelps to Eldritch Tentacle in zone Evoke Eldritch Horror, at "turn" 51
Error at Preference lastEncounter changed from Up In Their Grill to Centurion of Sparky in zone Infernal Seal

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,actualAdvs,freeTurns,banishes,GSBs,freeRuns,freeKills,saberCount,goblinCount,ensorcels,macros,odorExtractors,sniffs,enamorangs
location,,,,,,,,,,,,,
The Battlefield (Frat Uniform),90,3,0,0,0,0,0,0,0,0,0,0,0
The Penultimate Fantasy Airship,23,4,0,0,0,0,0,0,0,0,0,0,0
"The Arid, Extra-Dry Desert",17,4,0,0,0,0,0,0,0,0,0,0,0
The Smut Orc Logging Camp,12,0,0,0,0,0,0,0,0,0,0,0,0
McMillicancuddy's Barn,11,1,0,0,0,0,0,0,0,0,0,0,0
The Lower Chamber,10,0,0,0,0,0,0,0,0,0,0,0,0
The Castle in the Clouds in the Sky (Ground Floor),10,1,0,0,0,0,0,0,0,0,0,0,0
The Spooky Forest,10,3,0,0,0,0,0,0,0,0,0,0,0
The Black Forest,9,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
scotch377exploath = newLogParser(runDates=['20190823','20190824'],zInfo=zoneInfo,kolName='Captain Scotch').prepareNewLog()
resourcesByZone(scotch377exploath).to_csv('scotchExploath2.csv')
resourcesByZone(scotch377exploath)

Day #1 -- Parsing 20190823.
Error at the God Lobster in zone God Lobster, at "turn" 56
Error at Granted a Boon in zone God Lobster, at "turn" 57
Error at the God Lobster in zone God Lobster, at "turn" 95
Error at Granted a Boon in zone God Lobster, at "turn" 96
Error at Eldritch Tentacle in zone Evoke Eldritch Horror, at "turn" 97
Error at the God Lobster in zone God Lobster, at "turn" 132
Error at Granted a Boon in zone God Lobster, at "turn" 133
Error at War Hippy Infantryman in zone The Exploaded Battlefield, at "turn" 188
Error at War Hippy Homeopath in zone The Exploaded Battlefield, at "turn" 189
Error at War Hippy Homeopath in zone The Exploaded Battlefield, at "turn" 190
Error at War Hippy Baker in zone The Exploaded Battlefield, at "turn" 191
Error at War Hippy Rigger in zone The Exploaded Battlefield, at "turn" 192
Error at War Hippy Windtalker in zone The Exploaded Battlefield, at "turn" 194
Error at War Hippy Dread Squad in zone The Exploaded Battlefield, at "turn" 195
Erro

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,actualAdvs,freeTurns,banishes,GSBs,freeRuns,freeKills,saberCount,goblinCount,ensorcels,macros,odorExtractors,sniffs,enamorangs
location,,,,,,,,,,,,,
The Exploaded Battlefield,76,5,1,0,0,0,1,3,0,1,1,1,0
"The Arid, Extra-Dry Desert",21,6,0,0,0,3,0,3,0,0,0,0,0
The Black Forest,12,1,0,0,0,0,1,0,0,0,0,0,0
The Hidden Bowling Alley,10,2,0,0,0,0,0,0,0,0,0,0,0
The Smut Orc Logging Camp,10,0,0,0,0,0,0,0,0,0,0,0,0
The Lower Chamber,10,0,0,0,0,0,0,0,0,0,0,0,0
The Middle Chamber,9,2,1,0,0,0,0,1,0,0,0,0,0
Inside the Palindome,9,2,2,0,0,0,0,0,0,2,1,0,0
The Copperhead Club,8,7,1,2,0,0,0,3,0,0,0,0,0
